In [7]:
import pandas as pd
import numpy as np
import os
import math

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import pyspark.sql
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pyspark.sql.types import *
from pyspark.sql.functions import desc, asc
from pyspark.sql import functions as f
from pyspark.sql.functions import col

In [100]:
basic_folder = './results/'
file_list = os.listdir(basic_folder)

result_file = open('dataProcess_result.txt','w')


for file in file_list:
    file_name = basic_folder + file
    
    # 열 이름 추출
    df_pd = pd.read_csv(file_name)
    df_pd = df_pd.drop(['Unnamed: 0'],axis=1)
    
    # 스키마 정의
    devColumns = [StructField('index',IntegerType())]
    intCols = ['rank','category','price', 'discount_percentage', 'rating_total_count',
              'reviews_for_last1year']
    doubleCols = ['rating']
    boolCols = ['rocket_delivery', 'is_out_of_stock']
    for col in df_pd.columns:
        type_col = StringType()
        if col in intCols:
            type_col = IntegerType()
        elif col in doubleCols:
            type_col = DoubleType()
        elif col in boolCols:
            type_col = BooleanType()

        devColumns.append(StructField(col,type_col))

    schema = StructType(devColumns)
    
    # 스파크 dataframe 생성
    df = spark.read.schema(schema).csv(file_name, header=True)
    
    # 카테고리 출력
    item = file[:file.find('_')]
    print('카테고리 : ',item)
    
    # 판매량 산정
    temp = df_pd['reviews_for_last1year'].tolist()
    temp = temp[::-1]
    temp = [x for x in temp if math.isnan(x) == False]
    std = int(np.mean(temp[:10]))
    
    if item == "hanger,doorhook":
        df = df.withColumn('sales', (1080 - df.index)*std)
    else:
        df = df.withColumn('sales', (1080 - df.rank)*std)
    
    # 상품별 인기 요소
    keywords = []
    
    # 특성별 판매량 순위
    for col in df_pd.columns[13:]:
        #print('특성 :', col)
        result = df.groupby(col).agg(f.count(col).alias("개수"),
                                     f.sum('sales').alias('판매량')).sort(desc('판매량'))
        result = result.withColumn('상품당 판매량',(result.판매량/result.개수)).sort(desc('상품당 판매량'))
        result = result.filter((f.col('개수') > 9) & (f.col(col) != 'NULL'))
        
        if result.count() != 0:
            key = str(result.select(col).take(1)[0])
            key = key[key.find("=")+2:key.find(")")-1]
            keywords.append(col+ ' : ' +key)
        
        #result.show()
    print("인기 요소 :", keywords)
    print('-------------------------------------------------------------------------------------------------------------------')
    result_file.write('카테고리 : ' + item + '\n')
    result_file.write('인기 요소 : [')
    result_file.write(', '.join(keywords))
    result_file.write(']\n')

result_file.close()
    

카테고리 :  bedtray
인기 요소 : ['색상 : 브라운', '형태 : 직사각형', '인테리어 : 모던/심플', '재질 : 원목', '접이 가능여부 : 접이식가능']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  bed
인기 요소 : ['인테리어 : 모던/심플', '재질 : 합성목재', '사이즈 : 싱글', '설치 : 방문설치', '색상 : 블랙', '매트리스포함 : 매트리스포함', '침대 프레임 : 밀폐형']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  chair
인기 요소 : ['색상 : 베이지', '소재 : 플라스틱', '재질 : 철제', '팔걸이 : 팔걸이없음', '등받이 : 등받이없음', '종류 : 야외/캠핑용의자', '인테리어 : 모던/심플']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  closet
인기 요소 : ['색상 : 그레이', '재질 : PB + MDF', '인테리어 : 내추럴']
-------------------------------------------------------------------------------------------------------------------
카테고리 :  drawers
인기 요소 : ['색상 : 화이트', '재질 : 철제', '인테리어 : 내추럴']
--------------------------------------------------------------------

In [6]:
# 스파크 리셋
sc.stop()